In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import matplotlib.dates as mdates
import matplotlib.pylab as plt
import datetime as dt
import glob

In [ ]:
# Only considers dives where there was a GPS signal less then 3 mins before (and after?)
# the dive

# This code works, however I am moving forward with combining multiple csv files.

# Group and combine according to the tag number

# Need to figure out the GPS signal periods but the dtype is marked with useful
# variables:

# SEN_ACC_5Hz is dropped (to be used for waves but that's probably useless for now?)

# SEND_ALL2_1Hz_START
# How about datatypes: GPS, GPSS, GPSD

# This code requires the mounted directory: /mnt/UAE_data_.
# see README for details


# So there will be a GPS signal right before _START and after _END.

In [2]:
keepdat=["device_id","UTC_datetime","UTC_date","UTC_time","datatype","satcount","Latitude","Longitude","depth_m",
         "conductivity_mS/cm","ext_temperature_C"]

In [3]:
##### show all the columns of the Pandas DataFrame
pd.set_option('display.max_columns',None)

def pd_read_pattern(pattern):
    files = glob.glob(pattern)
 
    dfs = pd.DataFrame()
#     print(dfs)
    for f in files:
        
        print (f)
        
        dfs = dfs.append(pd.read_csv(f,parse_dates=True,index_col='UTC_datetime',
                          usecols=keepdat))
        
        dfs.drop(dfs[dfs['datatype'] == 'SEN_ACC_5Hz'].index, inplace = True)
        dfs.drop(dfs[dfs['datatype'] == 'SEN_ACC_5Hz_END'].index, inplace = True)
        dfs.drop(dfs[dfs['datatype'] == 'SEN_ACC_5Hz_START'].index, inplace = True)
        dfs.drop(dfs[dfs['datatype'] == 'SEN_ACC_5Hz_ENDINT'].index, inplace = True)

    return dfs #.reset_index(drop=True)

In [4]:
# get IDs observed in UAEBUS///
# idfs = pd.DataFrame()
idfile = '/home/server/pi/homes/liux8/work/mnt/Field_Data/DASHCAMS_Deployment_Field_Data.csv';
IDS = pd.read_csv(idfile,encoding='latin-1')
IDS.drop(IDS[IDS['Project_ID'] != 'UAEBUSO20'].index, inplace = True)
IDS.columns
# kuae = IDS(strcmp(IDS.Var4,'UAEBUSO20'),:);
# idnumber0 = [];
# for i = 1:size(kuae.Var14,1)
#    idnumber0 = kuae.Var14{i};
#    idnum{i} = idnumber0(:,end-5:end);
# end
# id_total = size(idnum,1);
# IDS['Deployment_ID'].shape[0]

Index(['ProjectTitle', 'PIEmail', 'DataManagerEmail', 'Project_ID', 'Bird_ID',
       'Species_Code', 'Species', 'Capture_Site', 'Capture_Number',
       'Capture_Date', 'Capture_Time', 'Release_Date', 'Release_Time',
       'Deployment_ID', 'DeploymentStartDatetime', 'UTC_offset_deploy',
       'DeploymentLatitude', 'DeploymentLongitude', 'TagManufacture',
       'TagModel', 'Sensors', 'TagSerialNumber', 'USGS_Band', 'Aux_Band_Code',
       'Aux_Band_Color', 'Tarsus_mm', 'Culmen_mm', 'Wing_Chord_cm',
       'Total_Weight_kg', 'Bag_Weight_kg', 'Bird_Weight_kg', 'Sex_Mass',
       'Feathers_YN', 'AnimalAgeClass', 'Plumage_BBL', 'Plumage_Notes',
       'Notes', 'DeploymentEndDatetime_UTC', 'Deployment_End_Notes',
       'Deployment_End_Short', 'DataEmbargoUntilDate', 'Status'],
      dtype='object')

In [6]:
dfs = pd.DataFrame()
for iid in range(IDS['Deployment_ID'].shape[0]):
# for iid in range(1):
    id = IDS['Deployment_ID'].values[iid][-6:]
    print(id)
    dfs = dfs.append(pd_read_pattern('/home/server/pi/homes/liux8/work/mnt/UAE_2020/'+id+'_2022_[0-1][0-9].csv'))
    if not dfs.empty:
        dfs=dfs.loc['2022-01-01':'2022-12-31']

        dfs['date']=pd.to_datetime(dfs['UTC_date']+dfs['UTC_time'], format='%Y-%m-%d%H:%M:%S')

        threshold = pd.Timedelta(minutes=3)
    print(dfs['device_id'][0:5])

203018
/home/server/pi/homes/liux8/work/mnt/UAE_2020/203018_2022_01.csv
/home/server/pi/homes/liux8/work/mnt/UAE_2020/203018_2022_02.csv
/home/server/pi/homes/liux8/work/mnt/UAE_2020/203018_2022_03.csv
/home/server/pi/homes/liux8/work/mnt/UAE_2020/203018_2022_04.csv
/home/server/pi/homes/liux8/work/mnt/UAE_2020/203018_2022_05.csv
/home/server/pi/homes/liux8/work/mnt/UAE_2020/203018_2022_06.csv
/home/server/pi/homes/liux8/work/mnt/UAE_2020/203018_2022_07.csv
/home/server/pi/homes/liux8/work/mnt/UAE_2020/203018_2022_08.csv
/home/server/pi/homes/liux8/work/mnt/UAE_2020/203018_2022_09.csv


ValueError: Usecols do not match columns, columns expected but not found: ['Longitude', 'conductivity_mS/cm', 'device_id', 'satcount', 'Latitude', 'datatype', 'UTC_time', 'depth_m', 'UTC_datetime', 'ext_temperature_C', 'UTC_date']

In [4]:
dfs = pd_read_pattern('/home/server/pi/homes/liux8/work/mnt/UAE_2020/*_2021_[0-1][1-9].csv')

dfs=dfs.loc['2021-01-01':'2021-12-31']

dfs['date']=pd.to_datetime(dfs['UTC_date']+dfs['UTC_time'], format='%Y-%m-%d%H:%M:%S')

threshold = pd.Timedelta(minutes=3)

/home/server/pi/homes/liux8/work/mnt/UAE_2020/201410_2021_01.csv
/home/server/pi/homes/liux8/work/mnt/UAE_2020/201410_2021_02.csv
/home/server/pi/homes/liux8/work/mnt/UAE_2020/201410_2021_03.csv
/home/server/pi/homes/liux8/work/mnt/UAE_2020/201411_2021_01.csv
/home/server/pi/homes/liux8/work/mnt/UAE_2020/201411_2021_02.csv
/home/server/pi/homes/liux8/work/mnt/UAE_2020/201411_2021_03.csv
/home/server/pi/homes/liux8/work/mnt/UAE_2020/201411_2021_04.csv
/home/server/pi/homes/liux8/work/mnt/UAE_2020/201411_2021_05.csv
/home/server/pi/homes/liux8/work/mnt/UAE_2020/201411_2021_06.csv
/home/server/pi/homes/liux8/work/mnt/UAE_2020/201411_2021_07.csv
/home/server/pi/homes/liux8/work/mnt/UAE_2020/201411_2021_08.csv
/home/server/pi/homes/liux8/work/mnt/UAE_2020/201418_2021_01.csv
/home/server/pi/homes/liux8/work/mnt/UAE_2020/201418_2021_02.csv
/home/server/pi/homes/liux8/work/mnt/UAE_2020/201418_2021_03.csv
/home/server/pi/homes/liux8/work/mnt/UAE_2020/201418_2021_04.csv
/home/server/pi/homes/liu

In [96]:
dfs.to_csv('surf_processed/All_2021_3min_raw_before_adjust.csv')

In [5]:
dfs['mask'] = dfs['date'].diff().abs().le(threshold)
dfs['Latitude_1']=dfs['Latitude'].shift(periods =-1 ,  fill_value='0')
dfs['Longitude_1']=dfs['Longitude'].shift(periods =-1 ,  fill_value='0')
dfs['mask_1']=dfs['mask'].shift(periods =-1 ,  fill_value='0')

dfs=dfs[dfs['datatype'] =='SEND_ALL2_1Hz_END']
dfs=dfs[dfs['mask_1'] == 1]
# dfs.to_csv('surf_processed/All_2021_3min_with_id.csv')
dfs.to_csv('surf_processed/All_2021_3min_with_id_older.csv')